In [1]:
import polars as pl
import pandas as pd
import re

In [ ]:
# We expect all genes to be represented in the GWAS Set, but here we double check to be sure

In [190]:
# Cleaned GFF3 from: ../ANALYSIS_SnpEff/ANALYSIS_SnpEff/updatedAnnotation/Clean_GCF_004027225.2_bStrHab1.2.pri_genomic.gff.gz
# Version: GCF_004027225.2_bStrHab1.2

In [191]:
gff3_columns = ["Landmark", "Source", "Type", "Start", "End", "Strand", "Attributes"]

gff3 = pl.read_csv("cleaned.gff3", comment_char = "#", separator="\t", has_header=False, new_columns=gff3_columns, columns=[0, 1, 2, 3, 4, 6, 8])
genes = gff3.filter(pl.col("Type") == "gene")
cds = gff3.filter(pl.col("Type") == "CDS")

# Get the gene ID
matcher = re.compile(r'GeneID:([\d]+)[,;]')

genes.head()

Landmark,Source,Type,Start,End,Strand,Attributes
str,str,str,i64,i64,str,str
"""S1""","""Gnomon""","""gene""",25222,29472,"""+""","""ID=gene-LOC115..."
"""S1""","""Gnomon""","""gene""",32484,39309,"""+""","""ID=gene-CCDC16..."
"""S1""","""Gnomon""","""gene""",41685,71311,"""+""","""ID=gene-MAPK15..."
"""S1""","""Gnomon""","""gene""",65399,90205,"""-""","""ID=gene-FAM83H..."
"""S1""","""Gnomon""","""gene""",110408,211712,"""-""","""ID=gene-SCRIB;..."


In [192]:
# Add Gene ID as a separate column

def extract(s):
    match = matcher.search(s)
    if match:
        return match.group(1)
    else:
        return ""

gene_ids = genes.apply(lambda r: extract(r[6]))
genes = genes.with_columns([
    (gene_ids[:, 0]).alias("ID"),
])
genes.head()

Landmark,Source,Type,Start,End,Strand,Attributes,ID
str,str,str,i64,i64,str,str,str
"""S1""","""Gnomon""","""gene""",25222,29472,"""+""","""ID=gene-LOC115...","""115603991"""
"""S1""","""Gnomon""","""gene""",32484,39309,"""+""","""ID=gene-CCDC16...","""115603988"""
"""S1""","""Gnomon""","""gene""",41685,71311,"""+""","""ID=gene-MAPK15...","""115603977"""
"""S1""","""Gnomon""","""gene""",65399,90205,"""-""","""ID=gene-FAM83H...","""115603962"""
"""S1""","""Gnomon""","""gene""",110408,211712,"""-""","""ID=gene-SCRIB;...","""115603994"""


In [193]:
matcher_name = re.compile(r'protein_id=(XP_[\w\d\.]+)[,;\s$]?')#[\w\d\._]+)[,;\$\s]')

def extract_names(s):
    match = matcher_name.search(s)
    if match:
        return match.group(1)
    else:
        return ""

cds_ids = cds.apply(lambda r: extract(r[6]))
names = cds.apply(lambda r: extract_names(r[6]))

cds = cds.with_columns([
    (cds_ids[:, 0]).alias("ID"),
    (names[:, 0]).alias("Name"),
])
cds.head()
cds_map = cds.select((pl.col("ID"), pl.col("Name")))
cds_map.head()

ID,Name
str,str
"""115603988""","""XP_030332433.1..."
"""115603977""","""XP_030332421.1..."
"""115603977""","""XP_030332421.1..."
"""115603977""","""XP_030332421.1..."
"""115603977""","""XP_030332421.1..."


In [194]:
# Using GWAS Input File gwas_snps_filtered_minobs_60_minaf_0.02.hapmap.txt.hmp.txt
snp_sites = pl.read_csv("/home/josephguhlin/development/kakapo-phenos/gwas_snps_filtered_minobs_60_minaf_0.02.hapmap.txt.hmp.txt", separator="\t", columns=[2, 3])

In [195]:
snp_sites[0:5]

chrom,pos
str,i64
"""NC_044298.1_CT...",666
"""NC_044298.1_CT...",714
"""NC_044298.1_CT...",822
"""NC_044298.1_CT...",1343
"""NC_044298.1_CT...",1802


In [205]:
found_genes = list()

with pl.StringCache():
    snp_sites = snp_sites.with_columns(pl.col("chrom").alias("Landmark").cast(pl.Categorical))
    genes = genes.with_columns(pl.col("Landmark").cast(pl.Categorical)).lazy()

    for _, pos, landmark in snp_sites.iter_rows():
        found = genes.filter(pl.col("Landmark") == landmark).filter(pl.col("Start") <= pos).filter(pos <= pl.col("End")).collect()
        if len(found) > 0:

            # Because sometimes genes overlap
            for i in found["ID"]:
                found_genes.append(i)

In [207]:
[len(set(found_genes)), len(genes.collect())]

[15644, 19046]

In [217]:
geneid_to_xp = {k: v for k, v in cds_map.iter_rows()}

In [230]:
found_proteins = list()

for i in set(found_genes):
    if i in geneid_to_xp:
        found_proteins.append(geneid_to_xp[i])

In [233]:
# Comparison to https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/004/027/225/GCF_004027225.2_bStrHab1.2.pri/GCF_004027225.2_bStrHab1.2.pri_protein.faa.gz
len(set(found_proteins))

13661

In [241]:
pl.DataFrame(data = pl.Series(found_proteins).alias("ID")).write_parquet("FoundProteins.parquet")

# Load and export to file in the style of
https://github.com/tanghaibao/goatools/blob/main/tests/data/small_population

In [2]:
found_proteins = pl.read_parquet("FoundProteins.parquet")['ID'].to_list()
with open("background_geneset.txt", "w") as fh:
    for i in found_proteins:
        fh.write(i)
        fh.write("\n")